In [ ]:
import pandas as pd
import os
from gm.api import *
import datetime as dt
set_token('9f9a31a3ba21ce1492ca28eb5beaa2243848909d')
# 采用掘金量化接口获取所有A股股票的信息
data_jj = get_instruments(
    exchanges = 'SZSE,SHSE',
    sec_types = 1,
    df = True,
    skip_suspended=True,
    skip_st=True, 
)
print(data_jj.head(3))

In [15]:
#获取交易日信息
trade_calendar = get_trading_dates_by_year(
    exchange='SHSE', 
    start_year=int(dt.datetime.now().year),
    end_year=int(dt.datetime.now().year),
    )

# 首先检查数据类型
print("trade_date列的数据类型:", trade_calendar['trade_date'].dtype)
print("前几行数据:", trade_calendar['trade_date'].head())

# 转换为datetime类型
trade_calendar['trade_date'] = pd.to_datetime(trade_calendar['trade_date'])

checked_day = dt.datetime.now().date()

# 判断今天是否是交易日
if checked_day not in trade_calendar['trade_date'].dt.date.values:
    # 如果不是交易日，则获取前一个交易日
    # 这里的逻辑也需要修正
    previous_trade_day = trade_calendar[trade_calendar['trade_date'].dt.date < checked_day]['trade_date'].max()
    checked_day = previous_trade_day.date() if pd.notna(previous_trade_day) else checked_day
else:
    # 如果是交易日，则获取今天的日期
    checked_day = checked_day
print("交易日:", checked_day)


trade_date列的数据类型: object
前几行数据: 0              
1    2025-01-02
2    2025-01-03
3              
4              
Name: trade_date, dtype: object
交易日: 2025-06-06


d:\anaconda3\envs\quant\Lib\site-packages\gm\api\ds_instrument.py:163: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame(data).applymap(_datetime_to_str)


In [19]:
stock_data_list = []
for _, row in data_jj.iterrows():
    symbol = row.symbol
    single_stock_data = history(symbol=symbol, 
            frequency='1d', 
            start_time=checked_day, 
            end_time=checked_day, 
            fields=None, 
            skip_suspended=True,
            fill_missing=None, 
            df=True)
    
    if not single_stock_data.empty:  # 确保有数据才添加
        stock_data_list.append(single_stock_data) 

# 合并所有数据
if stock_data_list:
    stock_data_pool = pd.concat(stock_data_list, ignore_index=True)
    print(f"获取了 {len(stock_data_list)} 只股票的数据")
    print(stock_data_pool)
else:
    print("没有获取到任何股票数据")
    
print(stock_data_pool)

获取了 5205 只股票的数据
           symbol frequency   open   high    low  close    volume  \
0     SZSE.002485        1d   3.26   3.39   3.26   3.39   9032982   
1     SZSE.300293        1d  20.78  20.96  20.50  20.56   5183200   
2     SZSE.002213        1d  15.67  16.49  15.20  16.49  21709450   
3     SZSE.002917        1d  13.39  13.86  13.30  13.59  14271850   
4     SZSE.300366        1d   8.10   8.18   7.90   7.92  38789719   
...           ...       ...    ...    ...    ...    ...       ...   
5200  SHSE.603076        1d  37.55  41.00  35.88  37.15  17338043   
5201  SHSE.688459        1d   9.68   9.70   9.64   9.67   1173199   
5202  SHSE.600633        1d  13.41  13.66  13.15  13.32  37961682   
5203  SHSE.601500        1d   4.57   4.63   4.56   4.59  15535942   
5204  SHSE.600918        1d   6.33   6.33   6.25   6.25  14677149   

              amount  pre_close  position                       bob  \
0     3.05598900e+07       3.23         0 2025-06-06 00:00:00+08:00   
1     1.07227

In [20]:
print(stock_data_pool.columns)

Index(['symbol', 'frequency', 'open', 'high', 'low', 'close', 'volume',
       'amount', 'pre_close', 'position', 'bob', 'eob'],
      dtype='object')
